## VQE for Ground State Optimization

### Building the electronic Hamiltonian

We specify the molecule we want to simulate by using the [XYZ File Format](https://en.wikipedia.org/wiki/XYZ_file_format) which describes the location of atoms in a molecule. 

This takes the following structure
```
<number of atoms>
comment line
<element> <X> <Y> <Z>
...
```
Where X, Y and Z are the cartesian coordinates in angstroms ($10^{-10}$ m).

In [ ]:
input = """2
Sample H2 molecule
H 0.3710 0.0 0.0
H -0.3710 0.0 0.0"""

To use this in our code, we parse using `xyz_parse`

In [ ]:
import xyz_parse
molecule = xyz_parse.Molecule.parse(input)
repr(molecule)

With `qml.qchem.molecular_hamiltonian`, which receives the atomic symbols of the molecule and the space coordinates; we get `H`, the Hamiltonian as a linear combination of Pauli operators, and `qubits`, the number of required qubits

In [ ]:
import pennylane as qml
from pennylane import numpy as np

# Setup hamiltonian
H, qubits = qml.qchem.molecular_hamiltonian(
    molecule.symbols,
    # Factor of 1.88973 to convert Angstrom to Bohr
    np.array(molecule.coordinates, dtype=np.float64) * 1.88973
)
print("Number of qubits = ", qubits)
print("The Hamiltonian is ", H)

We use a [minimal basis set](<https://en.wikipedia.org/wiki/STO-nG_basis_sets>)
to represent the [molecular orbitals](https://en.wikipedia.org/wiki/Molecular_orbital>). This means we use one qubit for each possible spin orbital. In this approximation, we have four spin orbitals (two for each electron) which defines the number of qubits.

### Implementing the VQE algorithm

We can use Pennylane's standard qubit simulator:

In [ ]:
dev = qml.device("default.qubit", wires=qubits)

#### Hartree-Fock State

It is a common approximation for the ground state of a molecule, in which electrons are placed in the lowest molecular orbitals.

`qml.qchem.hf_state` returns a list where each entry is a qubit. A 1 indicates an electron in that spin orbital, and a 0 no electron. This is the Jordan-Winger encoding. 

We need to define the quantum circuit that prepares the trial state of the
molecule (ansatz). We want to prepare states of the form,

$$
    \vert \Psi(\theta) \rangle = \cos(\theta/2)~|1100\rangle -\sin(\theta/2)~|0011\rangle,
$$

This formula represents a normalized quantum state as a superposition of basis states. Normalized means the total probability must sum to 1.

The Hartree-Fock State ($|1100\rangle$) is a good starting approximation for the electronic ground state of a molecule.  It indicates two electrons occupying the lowest energy molecular orbitals.

On the other hand, $|0011\rangle$ is the double excitation state, the electrons occupy the highest energy orbitals.

So $\vert \Psi(\theta) \rangle$ is a variational form that allows the wavefunction to be a combination of these basis states


To implement this using Pennylane, we use the
`hf_state` function to generate the vector representing the Hartree-Fock state.

In [ ]:
electrons = 2
hf = qml.qchem.hf_state(electrons, qubits)
print(hf)

The `hf` array is used by the `pennylane.BasisState` operation to initialize
the qubit register. We apply the `pennylane.DoubleExcitation` operation on the four qubits. The next step is to compute the expectation value of the molecular Hamiltonian in the trial state prepared by the circuit.
We do this using the `expval` function. The decorator syntax allows us to run the cost function as an executable QNode with the gate parameter $\theta$:

In [ ]:
@qml.qnode(dev)
def circuit(param, wires):
    qml.BasisState(hf, wires=wires)
    qml.DoubleExcitation(param, wires=[0, 1, 2, 3])
    return qml.expval(H)

We can now define our error function simply as the expected value calculated above:

In [ ]:
def cost_fn(param):
    return circuit(param, wires=range(qubits))

We initialize the circuit parameter $\theta$ to zero, meaning that we start
from the Hartree-Fock state.

In [ ]:
theta = np.array(np.pi / 2)

Here we use a basic gradient-descent optimizer.
We carry out the optimization over a maximum of 100 steps aiming to reach a
convergence tolerance of $10^{-6}$ for the value of the cost function.

In [ ]:
max_iterations = 200
convergence_tolerance = 1e-06
optimizer = qml.GradientDescentOptimizer(stepsize=0.03)

We create a list of energies to then represent them in a graph using matplotlib

In [ ]:
energies = []

And we run the algorithm iterating over the calculated energy values until we reach the optimal one

In [ ]:
for _ in range(max_iterations):
    theta, prev_energy = optimizer.step_and_cost(cost_fn, theta)
    energy = cost_fn(theta)
    energies.append(energy)
    if np.abs(energy - prev_energy) <= convergence_tolerance:
        break

print(f"Final value of the ground-state energy = {energy:.8f} Ha")
print(f"Optimal value of the circuit parameter = {theta:.4f}")

We plot the graph that shows the target energy and the ones calculated in each iteration

In [ ]:
import matplotlib.pyplot as plt

plt.plot(energies, "b", label="Energy")
plt.axhline(-1.137, linestyle="--", color="black", label="Target Energy")
plt.xlabel("Optimization Iterations")
plt.ylabel("Energy")
plt.title("Ground State Energy Calculation")
plt.legend()
plt.show()

The quantum circuit to prepare the trial state $\vert \Psi(\theta) \rangle$ is
schematically illustrated in the figure below.


In this figure, the gate $G^{2}$ corresponds to the
`pennylane.DoubleExcitation` operation, implemented in PennyLane
as a [Givens rotation](https://en.wikipedia.org/wiki/Givens_rotation>), which couples
the four-qubit states $\vert 1100 \rangle$ and $\vert 0011 \rangle$

In [ ]:
fig, ax = qml.draw_mpl(circuit)(1.2345,1.2345)

To submit our results to **aqora** we need to set the output variable in our notebook

In [ ]:
output = float(energy)